In [3]:
#csv 파일 읽어서 sql파일에 넣기

import pymysql
import pandas as pd

def load_iris_data():
    csv_file = './datascience/iris.csv'
    iris = pd.read_csv(csv_file)
    
    #db connection 생성
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment3')
    curs = conn.cursor(pymysql.cursors.DictCursor)

    #겹칠 경우
    drop_sql = """drop table if exists iris"""
    curs.execute(drop_sql)
    conn.commit()
    
    #직접 데이터프레임으로부터 테이블을 만들고 입력 하는 모듈
    import sqlalchemy
    database_username = 'root'
    database_password = 'taeyoon!'
    database_ip = 'localhost'
    database_name = 'datascienceassignment3'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.format(database_username,database_password,database_ip,database_name))
    
    #iris 테이블 생성 + 데이터 입력
    iris.to_sql(con=database_connection, name='iris',if_exists='replace')
load_iris_data()

In [6]:
import pymysql
import numpy as np

conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment3')
curs = conn.cursor(pymysql.cursors.DictCursor)

sql="select * from iris"
curs.execute(sql)

#fetchone이 메모리요소에서 보면 바람직하나 데이터양이 적어 fetchall사용
data=curs.fetchall()

#불러오는법
#print(data)

curs.close()
conn.close()

In [15]:
#딕셔너리 형태가 튜플 형태의 리스트로 바뀐다.
X=[(t['sepal_length'],t['sepal_width'],t['petal_length'],t['petal_width'])for t in data]
#print(X)


#print(type(X)) - <class 'list'>
#print(type(X[0])) - <class 'tuple'>

#tuple의 리스트가 numpy로 바뀐다.
X=np.array(X)
#print(X)

#print(X.shape) - (150, 4)

In [19]:
y=[1 if (t['species']=='versicolor') else -1 for t in data]
y=np.array(y)
#print(y)
#print(y.shape) - (150,)

In [126]:
#train data와 test data분리

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.33, random_state=42) #보통 42로 고정

#print(X_train)
#print(y_train)

from sklearn import tree
#모델 생성
dtree = tree.DecisionTreeClassifier()
dtree_model = dtree.fit(X_train, y_train) 

#예측 값
y_predict = dtree_model.predict(X_test)
print(y_predict)

[ 1 -1 -1  1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1  1  1 -1 -1  1  1 -1
  1 -1]


In [127]:
#성능 측정
def classification_performance_eval(y,y_predict):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for y, yp in zip(y,y_predict) :
            if (y == 1 and yp == 1):
                tp += 1
            elif(y == 1 and yp == -1):
                fn += 1
            elif(y== -1 and yp == 1) : 
                fp += 1
            else:
                tn += 1
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = (tp)/(tp+fp)
    recall = (tp) / (tp+fn)
    f1_score = 2*precision*recall / (precision + recall)
    
    return accuracy,precision,recall,f1_score

acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict)

print("accuracy=%f"%acc)
print("precision=%f"%prec)
print("recall=%f"%rec)
print("f1_score=%f"%f1)

accuracy=0.980000
precision=0.937500
recall=1.000000
f1_score=0.967742


In [133]:
from sklearn.model_selection import KFold

accuracy= []
precision = []
recall = []
f1_score = []

kf = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index,test_index in kf.split(X):
    X_train,X_test = X[train_index],X[test_index]
    y_train,y_test = y[train_index],y[test_index]
    
    dtree = tree.DecisionTreeClassifier()
    dtree = dtree_model.fit(X_train,y_train)
    y_predict = dtree_model.predict(X_test)
    acc, prec, rec, f1 = classification_performance_eval(y_test,y_predict) 
    accuracy.append(acc)
    precision.append(prec)
    recall.append(rec)
    f1_score.append(f1)
    
print(accuracy)

[1.0, 1.0, 0.9333333333333333, 0.9333333333333333, 0.9333333333333333]


In [135]:
import statistics

print("average_accuracy = ",statistics.mean(accuracy))
print("average_precision = ",statistics.mean(precision))
print("average_recall = ",statistics.mean(recall))
print("average_f1_score = ",statistics.mean(f1_score))

average_accuracy =  0.96
average_precision =  0.9492307692307692
average_recall =  0.9400000000000001
average_f1_score =  0.9411111111111111
